In [1]:
# Setup
import numpy as np
import os
import symforce
from symforce import typing as T

symforce.set_symbolic_api("symengine")
symforce.set_log_level("warning")

In [2]:
symforce.set_epsilon_to_symbol()

In [3]:
from symforce import codegen
from symforce.codegen import codegen_util
from symforce import ops
import symforce.symbolic as sf
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

In [4]:
# https://en.wikipedia.org/wiki/Double_integrator
q = sf.V3.symbolic("q")
display(q)

⎡q₀⎤
⎢  ⎥
⎢q₁⎥
⎢  ⎥
⎣q₂⎦

In [5]:
continuous_time_dynamics = sf.V2([q[1], q[2]])
display(continuous_time_dynamics)

⎡q₁⎤
⎢  ⎥
⎣q₂⎦

In [6]:
state = sf.V2([q[0], q[1]])

In [7]:
u = sf.V1(q[2])

In [8]:
# def double_integrator_discrete_time_dynamics(
#     x:sf.Vector2, u:sf.Vector1
# ) -> sf.Vector2:
#     A = continuous_time_dynamics.jacobian(x)
#     B = continuous_time_dynamics.jacobian(u)
#     qdot = sf.V2(A*x + B*u)
#     return qdot

In [9]:

# def rk4(x:sf.Vector2, u:sf.Vector1, dt:sf.Scalar) -> sf.Vector2: 
#     # vanilla RK4
#     k1 = dt * double_integrator_discrete_time_dynamics(x, u)
#     k2 = dt * double_integrator_discrete_time_dynamics(x + k1/2, u)
#     k3 = dt * double_integrator_discrete_time_dynamics(x + k2/2, u)
#     k4 = dt * double_integrator_discrete_time_dynamics(x + k3, u)
#     xplus1 =  (x + (1/6)*(k1 + 2*k2 + 2*k3 + k4))
#     return xplus1


In [10]:
inputs = Values(
    state = sf.V2([q[0], q[1]]),
    u = sf.V1(q[2]),
)

In [11]:
# A = continuous_time_dynamics.jacobian(state)
# B = continuous_time_dynamics.jacobian(u)

In [12]:
outputs = Values(
    qdot = sf.V2(continuous_time_dynamics.jacobian(state)*state + continuous_time_dynamics.jacobian(u)*u)
    )

In [14]:
discrete_time_dynamics_codegen = codegen.Codegen(
    inputs=inputs,
    outputs=outputs,
    config=codegen.CppConfig(),
    name="double_integrator_discrete_time_dynamics",
    return_key="qdot",
)
discrete_time_dynamics_data = discrete_time_dynamics_codegen.generate_function()

# Print what we generated
print("Files generated in {}:\n".format(discrete_time_dynamics_data.output_dir))
for f in discrete_time_dynamics_data.generated_files:
    print("  |- {}".format(os.path.relpath(f, discrete_time_dynamics_data.output_dir)))

Files generated in /tmp/sf_codegen_double_integrator_discrete_time_dynamics_t_c84j20:

  |- cpp/symforce/sym/double_integrator_discrete_time_dynamics.h
